# Retrieve, Process and Save historic data ETL

This file contains the ETL of retrieve raw data from S3, process them and save back to S3.
Note that this is developed in PySpark enviroment.

## Session 1 load package and and moudles, Start Spark Session.

In [1]:
from pyspark.sql import SparkSession
import os
import configparser



VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1590452981392_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# config = configparser.ConfigParser()

# #Normally this file should be in ~/.aws/credentials
# config.read_file(open('aws/credentials.cfg'))

# os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
# os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

In [2]:
 spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sc.install_pypi_package("boto3") 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e4/be/3324ee30cce696a8798f9a3cb5acdb4090cda7075fec0b5ef2e9f8e220dd/botocore-1.16.17-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e1/e5/df302e8017440f111c11cc41a6b432838672f5a70aa29227bf58149dc72f/urllib3-1.25.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/cd/a6aa959dca619918ccb55023b4cb151949c64d4d5d55b3f4ffd7eee0c6e8/docutils-0.15.2-py3-none-any.whl

In [7]:
sc.install_pypi_package("pandas")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/4a/6a/94b219b8ea0f2d580169e85ed1edc0163743f55aaeca8a44c2e8fc1e344e/pandas-1.0.3-cp37-cp37m-manylinux1_x86_64.whl

In [137]:
sc.install_pypi_package("s3fs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/b8/e4/b8fc59248399d2482b39340ec9be4bb2493846ac23641b43115a7e5cd675/s3fs-0.4.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/31/f27a81686b2f1b2f6776bd5db10efc7d88f28a50e8888f55409ef6501a50/fsspec-0.7.4-py3-none-any.whl

# Session 2 Obatain historical data from S3 zip files

In [10]:
import zipfile
import boto3
import io
import os
import pandas as pd

def convert_zip_to_dfs(bucket_name, key):
    s3_resource = boto3.resource('s3')
    zip_obj = s3_resource.Object(bucket_name=bucket_name, key=key)
    buffer = io.BytesIO(zip_obj.get()["Body"].read())
    z = zipfile.ZipFile(buffer)

    # https://stackoverflow.com/questions/44575251/reading-multiple-files-contained-in-a-zip-file-with-pandas
    dfs = {os.path.splitext(text_file.filename)[0]: pd.read_csv(z.open(text_file.filename))
           for text_file in z.infolist()
           if text_file.filename.endswith('.csv')}
    return dfs


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Compnayinfo only has one file, only generated one dataframe [4610 rows x 33 columns]

In [97]:
df_companyinfo_list= convert_zip_to_dfs("stock.etl", "raw-historic-data/companyinfo.zip")
df_companyinfo_key=list(df_companyinfo_list.keys())[0]
df_company_info=df_companyinfo_list.get(df_companyinfo_key)
df_company_info

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                   simple_name  ... year_founded
0              Aurora Cannabis  ...       2006.0
1                         Ford  ...       1903.0
2                           GE  ...       1878.0
3                    Microsoft  ...       1975.0
4                        GoPro  ...       2002.0
...                        ...  ...          ...
4605              Regal Beloit  ...       1955.0
4606              RBC Bearings  ...       1919.0
4607                    ATRION  ...       1944.0
4608  Wealthbridge Acquisition  ...       2018.0
4609             AIRT Warrants  ...       1980.0

[4610 rows x 33 columns]

News have three files, generated three dataframes, 'CombinedNewsDJIA.csv' is a combined version of 'RedditNews.csv'.
The first column is "Date", the second is "Label", and the following ones are news headlines ranging from "Top1" to "Top25".
Here I use 'Combined_News_DJIA.csv' as df_news_historic, which has [1989 rows x 27 columns]

In [11]:
df_news_list= convert_zip_to_dfs("stock.etl", "raw-historic-data/stocknews.zip")
df_news_historic=df_news_list.get('Combined_News_DJIA')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_news_historic

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

            Date  ...                                              Top25
0     2008-08-08  ...           b"No Help for Mexico's Kidnapping Surge"
1     2008-08-11  ...  b"So this is what it's come to: trading sex fo...
2     2008-08-12  ...  b"BBC NEWS | Asia-Pacific | Extinction 'by man...
3     2008-08-13  ...  b'2006: Nobel laureate Aleksander Solzhenitsyn...
4     2008-08-14  ...  b'Philippines : Peace Advocate say Muslims nee...
...          ...  ...                                                ...
1984  2016-06-27  ...  Pakistani clerics declare transgender marriage...
1985  2016-06-28  ...  Mexico's security forces routinely use 'sexual...
1986  2016-06-29  ...  Tiny New Zealand town with 'too many jobs' lau...
1987  2016-06-30  ...  Law Enforcement Sources: Gun Used in Paris Ter...
1988  2016-07-01  ...  Ozone layer hole seems to be healing - US &amp...

[1989 rows x 27 columns]

stock price contains subfolders : "Data/ETFs/" and "Data/Stocks/". Files are in 'txt' format. Here I only consider stocks. There are a total of 7163 stocks. Each stock has a file, contains [8364 rows x 7 columns]. So the total number of rows for all stock files are about 6 million lines of data

In [13]:
def convert_stock_from_zip(bucket_name, key):
    s3_resource = boto3.resource('s3')
    zip_obj = s3_resource.Object(bucket_name=bucket_name, key=key)
    buffer = io.BytesIO(zip_obj.get()["Body"].read())
    z = zipfile.ZipFile(buffer)
    stock_dict_historic={}

    for file_info in z.infolist():
        if "Stocks" in file_info.filename and "txt" in file_info.filename:
            stock_name=file_info.filename.split("/")[1]
            stock_name=stock_name.split(".")[0]
            # some files has no data, need to exclude
            if stock_name !="Stocks" and file_info.file_size >0:
                df=pd.read_csv(z.open(file_info.filename), sep=',')
                stock_dict_historic.update({stock_name: df})
    return stock_dict_historic

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
stock_dict_historic=convert_stock_from_zip(bucket_name="stock.etl", key="raw-historic-data/stockpricing.zip")
stock_dict_historic.keys()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['a', 'aa', 'aaap', 'aaba', 'aac', 'aal', 'aamc', 'aame', 'aan', 'aaoi', 'aaon', 'aap', 'aapl', 'aat', 'aau', 'aav', 'aaww', 'aaxn', 'ab', 'abac', 'abax', 'abb', 'abbv', 'abc', 'abcb', 'abcd', 'abco', 'abdc', 'abe', 'abeo', 'abeow', 'abev', 'abg', 'abil', 'abio', 'ablx', 'abm', 'abmd', 'abr', 'abr_a', 'abr_b', 'abr_c', 'abrn', 'abt', 'abtx', 'abus', 'abx', 'aby', 'ac', 'acad', 'acbi', 'acc', 'acco', 'acer', 'acerw', 'acet', 'acfc', 'acgl', 'acglo', 'acglp', 'ach', 'achc', 'achn', 'achv', 'acia', 'aciu', 'aciw', 'acls', 'acm', 'acmr', 'acn', 'acnb', 'acor', 'acp', 'acre', 'acrs', 'acrx', 'acsf', 'acsi', 'acst', 'acta', 'actg', 'acu', 'acv', 'acxm', 'acy', 'adap', 'adbe', 'adc', 'ades', 'adhd', 'adi', 'adm', 'adma', 'admp', 'adms', 'adnt', 'adom', 'adp', 'adro', 'ads', 'adsk', 'adsw', 'adtn', 'adus', 'advm', 'adx', 'adxs', 'adxsw', 'ae', 'aeb', 'aed', 'aee', 'aeg', 'aegn', 'aeh', 'aehr', 'aeis', 'aek', 'ael', 'aem', 'aemd', 'aeo', 'aep', 'aer', 'aeri', 'aes', 'aet', 'aeti', 'ae

In [15]:
# number of stocks
len(stock_dict_historic.keys()) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7163

Here use APPLE.inc stock "AAPL" as an exmple to show the dataframe structure. 

In [22]:
#Example of a stock table 
stock_dict_historic.get('aapl')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

            Date       Open       High        Low      Close    Volume  OpenInt
0     1984-09-07    0.42388    0.42902    0.41874    0.42388  23220030        0
1     1984-09-10    0.42388    0.42516    0.41366    0.42134  18022532        0
2     1984-09-11    0.42516    0.43668    0.42516    0.42902  42498199        0
3     1984-09-12    0.42902    0.43157    0.41618    0.41618  37125801        0
4     1984-09-13    0.43927    0.44052    0.43927    0.43927  57822062        0
...          ...        ...        ...        ...        ...       ...      ...
8359  2017-11-06  171.75000  174.36000  171.10000  173.63000  34901241        0
8360  2017-11-07  173.29000  174.51000  173.29000  174.18000  24424877        0
8361  2017-11-08  174.03000  175.61000  173.71000  175.61000  24451166        0
8362  2017-11-09  174.48000  175.46000  172.52000  175.25000  29533086        0
8363  2017-11-10  175.11000  175.38000  174.27000  174.67000  25130494        0

[8364 rows x 7 columns]

# Session 3 Data Transformation and process

In this step, I will process and combine news, company info and stock pricing data. The reason to combine them to a single dataframe is to make is easier for training machine learning models. The historic data can be split into training and test datasets, using the combined table. Since here each stock are evaluated separatedly, each stock has an individual table. 


First step is to add a few stock technical indicators, such as bollinger bands, moving average, rate of change. Also calcuated daily return and a binary indicator for stock rise (1) or drop (or stay the same) (0). Missing values are taken care of by forward fill. 

It needs to be pointed out that the time for data collected of news and stock data are different, some stock data can trace back to 1980s and till 2017, but news data are since 2008 to 2016. Here I use inner join to avoid missing values. Also the moving average generated missing values at first N rows. Since they are at the beginning, was also taken care of by join the tables

In [81]:
import numpy as np
# Bollinger Bands
# https://www.kaggle.com/selval/analysing-stock-data
def get_bbands(df, ndays):
    dm = df[['Close']].rolling(ndays).mean()
    ds = df[['Close']].rolling(ndays).std()
    df['upperBB'] = dm + 2 * ds
    df['lowerBB'] = dm - 2 * ds
    return df

# Simple Moving Average
def get_SMA(df, ndays):
    df['SMA']=df[['Close']].rolling(ndays).mean()
    return df

# Expontential Moving Average
def get_EMA(df, ndays):
    df['EMA'] = df[['Close']].ewm( span = ndays, min_periods = ndays - 1).mean()
    return df

# Rate of Change
def get_ROC(df, ndays):
    dn = df[['Close']].diff(ndays)
    dd = df[['Close']].shift(ndays)
    df['ROC'] = dn/dd
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
def data_transformation(company_df, news_df, stock_df, stock_symbol):
    # First Keep useful information from stock price tables (Date, close and volume), forward fill missing values. 
    stock_price_df=stock_df.copy()
    stock_price_df.drop(['Open','High','Low', 'OpenInt'], axis=1, inplace=True)
    stock_price_df.fillna(method='ffill')
    stock_price_df.set_index('Date', inplace=True)
    
    
    # Adding daily return (%), stock_rise (1, 0 if drop), technical indicators 
    stock_price_df['daily_return']=stock_price_df[['Close']][:-1].values/stock_price_df[['Close']][1:]-1
    stock_price_df=get_bbands(stock_price_df, 10)
    stock_price_df=get_SMA(stock_price_df, 10)
    stock_price_df=get_EMA(stock_price_df, 10)
    stock_price_df=get_ROC(stock_price_df, 1)
    stock_price_df['stock_rise']=np.where(stock_price_df['daily_return']>0, 1, 0)
    
    
    # Adding company info
    company_info_df=company_df.copy()
    company_info_df.drop(['bloomberg_unique','margin_initial_ratio','maintenance_ratio','day_trade_ratio', 'list_date', 'default_collar_fraction', 'open', 'high', 'low', 'volume', 'average_volume_2_weeks', 'average_volume', 'high_52_weeks', 'dividend_yield', 'float', 'low_52_weeks', 'market_cap', 'pb_ratio', 'pe_ratio', 'shares_outstanding', 'description'], axis=1, inplace=True)
    row=company_info_df.loc[company_info_df['symbol'] == stock_symbol]
    if(len(row)>0):
        stock_price_df['simple_name']=row['simple_name'].values[0]
        stock_price_df['name']=row['name'].values[0]
        stock_price_df['country']=row['country'].values[0]
        stock_price_df['headquarters_city']=row['headquarters_state'].values[0]
        stock_price_df['sector']=row['sector'].values[0]
        stock_price_df['industry']=row['industry'].values[0]

    # Adding news data
    news_df_copy=news_df.copy()
    news_df_copy.drop(['Label'], axis=1, inplace=True)
    news_df_copy.set_index('Date', inplace=True)
    df_merge=pd.merge(stock_price_df, news_df_copy, left_index=True, right_index=True)
    
    return df_merge

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Also, use apple as an example to show the table after transformation and joined

In [128]:
df=data_transformation(df_company_info, df_news_historic, stock_dict_historic.get('aapl'), 'AAPL')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
pd.set_option('max_columns', None)
df.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

             Close     Volume  daily_return    upperBB    lowerBB      SMA  \
Date                                                                         
2008-08-08  21.711  199068558     -0.035143  21.730967  19.201433  20.4662   
2008-08-11  22.227  248378005     -0.023215  22.291594  19.131606  20.7116   
2008-08-12  22.633  233132531     -0.017938  22.886399  19.040001  20.9632   
2008-08-13  22.962  234899860     -0.014328  23.468691  18.955509  21.2121   
2008-08-14  22.963  198233593     -0.000044  23.886700  19.058900  21.4728   

                  EMA       ROC  stock_rise simple_name  \
Date                                                      
2008-08-08  20.813671  0.036424           0       Apple   
2008-08-11  21.070640  0.023767           0       Apple   
2008-08-12  21.354706  0.018266           0       Apple   
2008-08-13  21.646941  0.014536           0       Apple   
2008-08-14  21.886224  0.000044           0       Apple   

                               name cou

In [133]:
pd.set_option('max_columns', None)
df.tail(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

             Close    Volume  daily_return    upperBB    lowerBB      SMA  \
Date                                                                        
2016-06-27  89.635  45491756      0.014782  96.485221  89.639179  93.0622   
2016-06-28  91.145  39565203     -0.016567  96.031235  89.339365  92.6853   
2016-06-29  91.934  32943084     -0.008582  95.500221  89.336779  92.4185   
2016-06-30  93.102  34702639     -0.012545  94.794376  89.663024  92.2287   
2016-07-01  93.384  25603482     -0.003020  94.928414  89.637986  92.2832   

                  EMA       ROC  stock_rise simple_name  \
Date                                                      
2016-06-27  92.745937 -0.014567           1       Apple   
2016-06-28  92.454857  0.016846           0       Apple   
2016-06-29  92.360156  0.008657           0       Apple   
2016-06-30  92.495037  0.012705           0       Apple   
2016-07-01  92.656666  0.003029           0       Apple   

                               name country he

# Session 4 Save processed dataframes back to S3 indivisual Stock folder

This step is to save the processed dataframe as files to S3, each with name of the stock symbol. At first, I saved them to csv. But I encountered some issue with Pandas with Python2 not compatible with save to csv method on EC2 machine (though I can download python3, but it messed up airflow). So I ended up save files to json, which also has the benefit of smaller size. Below are two version of the methods.

In [ ]:
# save to S3
def save_to_s3(df, stock, folder):
    df_copy=df.copy()
    df_copy.reset_index(inplace=True)
    s3 = boto3.resource('s3')
    json_data=df_copy.to_json(orient='records')
    key="{}/{}.json".format(folder, stock)
    s3object = s3.Object('stock.etl', key)
    s3object.put(Body=(bytes(json.dumps(json_data).encode('UTF-8'))))

In [145]:
# save to S3-2
import s3fs

def save_to_s3_v2(df, stock, folder):
    key="{}/{}.csv".format(folder, stock)
    df.to_csv(os.path.join("s3a://stock.etl/", key), header=True, index=True, encoding='utf-8')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
save_to_s3(stock="AAPL", folder="historic.combine")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Main function

Before ran this main function, I made sure that single files read, process, and write working correctly to S3. Then run the ETL on airflow. Copied and modifed the code to DAG file (removed Spark). It is possible to run with Spark on EMR cluster, but since the data only run once, and it can finshied in acceptable time, I didn't use ERM with Spark.

In [4]:
def create_spark_session():
    """
    Create Spark session
    """
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
def main():
    spark = create_spark_session()
    
    # get company info
    df_companyinfo_list= convert_zip_to_dfs("stock.etl", "raw-historic-data/companyinfo.zip")
    df_companyinfo_key=list(df_companyinfo_list.keys())[0]
    df_company_info=df_companyinfo_list.get(df_companyinfo_key)

    # get news info
    df_news_list= convert_zip_to_dfs("stock.etl", "raw-historic-data/stocknews.zip")
    df_news_historic=df_news_list.get('Combined_News_DJIA')
    
    # get stock data
    stock_dict_historic=convert_stock_from_zip(bucket_name="stock.etl", key="raw-historic-data/stockpricing.zip")
    
    # process and save
    
    for key, value in stock_dict_historic.items():
        stock_name=key.upper()
        df=data_transformation(df_company_info, df_news_historic, value, stock_name)
        save_to_s3(df, stock=stock_name, folder="historic.combine")

if __name__ == "__main__":
    main()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…